# Exercises

This is the notebook containing the exercises for Feature Store, Model Monitor, and Clarify. Tested for these exercises was performed using __2 vCPU + 4 GiB notebook instance with Python 3 (TensorFlow 2.1 Python 3.6 CPU Optimized) kernel__.

## Staging

We'll begin by initializing some variables. These are often assumed to be present in code samples you'll find in the AWS documenation.

In [1]:
import sagemaker
from sagemaker.session import Session
from sagemaker import get_execution_role

role = get_execution_role()
session = sagemaker.Session()
region = session.boto_region_name
bucket = session.default_bucket()

/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pydantic/_internal/_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[03/29/25 04:42:28] INFO     Found credentials from IAM Role:                                   ]8;id=142521;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=174540;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/ec2-user/.config/sagemaker/config.yaml


[03/29/25 04:42:35] INFO     Found credentials from IAM Role:                                   ]8;id=45326;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=249037;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

[03/29/25 04:42:36] INFO     Found credentials from IAM Role:                                   ]8;id=901262;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=346304;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

## Feature Store
---

Feature store is a special database to give ML systems a consistent data flow across training and inference workloads. It can ingest data in batches (for training) as well as serve input features to models with very low latency for real-time prediction.

For this exercise we'll work with a wine quality dataset: https://archive.ics.uci.edu/ml/datasets/wine+quality/

```P. Cortez, A. Cerdeira, F. Almeida, T. Matos and J. Reis.
Modeling wine preferences by data mining from physicochemical properties. In Decision Support Systems, Elsevier, 47(4):547-553, 2009.```

In [2]:
import pandas as pd
from sklearn import datasets
import time
import uuid

In [3]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']

If we leave the column names as-is, Feature Store won't be able to handle the `/` in `od280/od315_of_diluted_wines` (`/` is a delimiter Feature Store uses to manage how features are organized.)

In [4]:
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

Once we have our data, we can create a feature group. Remember to attach event time and ID columns - Feature Store needs them.

In [5]:
# Add a feature group
df["EventTime"] = time.time()
df["id"] = range(len(df))

# TODO: Create feature group
from sagemaker.feature_store.feature_group import FeatureGroup

feature_group = FeatureGroup(
    name="wine-features", sagemaker_session=session
)

# TODO: Load Feature definitions
feature_group.load_feature_definitions(data_frame=df)

[FeatureDefinition(feature_name='alcohol', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='malic_acid', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='alcalinity_of_ash', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='magnesium', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='total_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='flavanoids', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),
 FeatureDefinition(feature_name='nonflavanoid_phenols', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collec

The feature group is not created until we call the `create` method, let's do that now:

In [7]:
# Create the feature store:

'sagemaker-us-east-1-510096416228'

In [6]:
# TODO
feature_group.create(
    s3_uri=f"s3://{bucket}/features",
    record_identifier_name='id',
    event_time_feature_name="EventTime",
    role_arn=role,
    enable_online_store=True,
)

{'FeatureGroupArn': 'arn:aws:sagemaker:us-east-1:510096416228:feature-group/wine-features',
 'ResponseMetadata': {'RequestId': 'dc68ab33-4674-4238-a4cf-a00d460c9739',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': 'dc68ab33-4674-4238-a4cf-a00d460c9739',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '90',
   'date': 'Sat, 29 Mar 2025 04:45:11 GMT'},
  'RetryAttempts': 0}}

Lastly, ingest some data into your feature group:

In [8]:
# TODO
feature_group.ingest(data_frame=df, max_workers=3, wait=True)

IngestionManagerPandas(feature_group_name='wine-features', feature_definitions={'alcohol': {'FeatureName': 'alcohol', 'FeatureType': 'Fractional'}, 'malic_acid': {'FeatureName': 'malic_acid', 'FeatureType': 'Fractional'}, 'ash': {'FeatureName': 'ash', 'FeatureType': 'Fractional'}, 'alcalinity_of_ash': {'FeatureName': 'alcalinity_of_ash', 'FeatureType': 'Fractional'}, 'magnesium': {'FeatureName': 'magnesium', 'FeatureType': 'Fractional'}, 'total_phenols': {'FeatureName': 'total_phenols', 'FeatureType': 'Fractional'}, 'flavanoids': {'FeatureName': 'flavanoids', 'FeatureType': 'Fractional'}, 'nonflavanoid_phenols': {'FeatureName': 'nonflavanoid_phenols', 'FeatureType': 'Fractional'}, 'proanthocyanins': {'FeatureName': 'proanthocyanins', 'FeatureType': 'Fractional'}, 'color_intensity': {'FeatureName': 'color_intensity', 'FeatureType': 'Fractional'}, 'hue': {'FeatureName': 'hue', 'FeatureType': 'Fractional'}, 'od280_od315_of_diluted_wines': {'FeatureName': 'od280_od315_of_diluted_wines', 'F

Great job! You've demonstrated your understanding of creating feature groups and ingesting data into them using Feature Store. Next up we'll cover Model Monitor!

## Model Monitor

In this exercise we'll create a monitoring schedule for a deployed model. We're going to provide code to help you deploy a model and get started, so that you can focus on Model Monitor for this exercise. __Remember to clean up your model before you end a work session__. We'll provide some code at the end to help you clean up your model. We'll begin by reloading our data from the previous exercise.



In [9]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [10]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [12]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

In [13]:
algo_image = sagemaker.image_uris.retrieve("xgboost", region, version='latest')
s3_output_location = f"s3://{bucket}/models/wine_model"

model=sagemaker.estimator.Estimator(
    image_uri=algo_image,
    role=role,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    volume_size=5,
    output_path=s3_output_location,
    sagemaker_session=sagemaker.Session()
)

model.set_hyperparameters(max_depth=5,
                        eta=0.2,
                        gamma=4,
                        min_child_weight=6,
                        subsample=0.8,
                        objective='reg:linear',
                        early_stopping_rounds=10,
                        num_round=200)


model.fit({'train': s3_input_train, 'validation': s3_input_validation})

[03/29/25 04:57:15] INFO     Ignoring unnecessary instance type: None.                            ]8;id=990970;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=85848;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

                    INFO     Found credentials from IAM Role:                                   ]8;id=730769;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=187248;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     Found credentials from IAM Role:                                   ]8;id=958696;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py\credentials.py]8;;\:]8;id=947448;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/botocore/credentials.py#1132\1132]8;;\
                             BaseNotebookInstanceEc2InstanceRole                                                   

                    INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=824413;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=418905;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/telemetry/telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

                    INFO     Creating training-job with name: xgboost-2025-03-29-04-57-15-927       ]8;id=169927;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=494258;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1042\1042]8;;\

2025-03-29 04:57:17 Starting - Starting the training job...
..25-03-29 04:57:32 Starting - Preparing the instances for training.
..25-03-29 04:57:57 Downloading - Downloading input data.
....Arguments: trainng - Downloading the training image.
[2025-03-29:04:59:34:INFO] Running standalone xgboost training.
[2025-03-29:04:59:34:INFO] File size need to be processed in the node: 0.01mb. Available memory size in the node: 8556.61mb
[2025-03-29:04:59:34:INFO] Determined delimiter of CSV input is ','
[04:59:34] S3DistributionType set as FullyReplicated
[04:59:34] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/train?format=csv&label_column=0&delimiter=,
[2025-03-29:04:59:34:INFO] Determined delimiter of CSV input is ','
[04:59:34] S3DistributionType set as FullyReplicated
[04:59:34] 89x13 matrix with 1157 entries loaded from /opt/ml/input/data/validation?format=csv&label_column=0&delimiter=,
[04:59:34] src/tree/updater_prune.cc:74: tree pruning end, 1 roots, 2 extra nodes, 0 pr

Now that your training job has finished, you can perform the first task in this exercise: creating a data capture config. Configure your model to sample `34%` of inferences:

In [27]:
# TODO
from sagemaker.model_monitor import DataCaptureConfig

capture_uri = f's3://{bucket}/data-capture'

data_capture_config = DataCaptureConfig(
    enable_capture=True,
    sampling_percentage=34,
    destination_s3_uri=capture_uri
)

Great! We'll use your config to deploy a model below:

In [ ]:
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)

[03/29/25 06:31:56] INFO     Creating model with name: xgboost-2025-03-29-06-31-56-199              ]8;id=584745;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=42644;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4094\4094]8;;\

[03/29/25 06:31:57] INFO     Creating endpoint-config with name xgboost-2025-03-29-06-31-56-199     ]8;id=667480;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=546360;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#5937\5937]8;;\

                    INFO     Creating endpoint with name xgboost-2025-03-29-06-31-56-199            ]8;id=589483;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=381429;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#4759\4759]8;;\

Great! You should see an indicator like this when the deployment finishes:

```
-----------------!
```
We can test your deployment like so:

In [16]:
xgb_predictor.serializer = sagemaker.serializers.CSVSerializer()
inputs = test.copy()
# Drop the target variable
inputs = inputs.drop(columns=inputs.columns[0])
x_pred = xgb_predictor.predict(inputs.sample(5).values).decode('utf-8')
x_pred

'0.6030303239822388,0.6030303239822388,0.6030303239822388,0.6030303239822388,0.7861111164093018'

All systems go! To finish up the exercise, we're going to provide you with a DefaultModelMonitor and a suggested baseline. Combine the `xgb_predictor` and the provided `my_monitor` to configure the monitoring schedule for _hourly_ monitoring.

In [17]:
from sagemaker.model_monitor import DefaultModelMonitor
from sagemaker.model_monitor.dataset_format import DatasetFormat

my_monitor = DefaultModelMonitor(
    role=role,
    instance_count=1,
    instance_type='ml.m5.xlarge',
    volume_size_in_gb=20,
    max_runtime_in_seconds=3600,
)

[03/29/25 05:07:28] INFO     Ignoring unnecessary instance type: None.                            ]8;id=839344;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=108537;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

In [18]:
my_monitor.suggest_baseline(
    baseline_dataset=f's3://{bucket}/data/train.csv',
    dataset_format=DatasetFormat.csv(header=False),
)

[03/29/25 05:08:04] INFO     Creating processing-job with name                                      ]8;id=11446;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py\session.py]8;;\:]8;id=185776;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/session.py#1575\1575]8;;\
                             baseline-suggestion-job-2025-03-29-05-08-04-003                                       

.............2025-03-29 05:10:13.805347: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2025-03-29 05:10:13.805377: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2025-03-29 05:10:15.310008: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2025-03-29 05:10:15.310036: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2025-03-29 05:10:15.310054: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-10-2-209-61.ec2.internal): /proc/driver/nvidia/version does not exist
2025-03-29 05:10:15.310308: I tensor

Below, provide the monitoring schedule:

In [19]:
# TODO
from sagemaker.model_monitor import CronExpressionGenerator

my_monitor.create_monitoring_schedule(
    monitor_schedule_name='wine-monitoring-schedule',
    endpoint_input=xgb_predictor.endpoint_name,
    statistics=my_monitor.baseline_statistics(),
    constraints=my_monitor.suggested_constraints(),
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

[03/29/25 05:13:29] INFO     Creating Monitoring Schedule with name:                       ]8;id=162411;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model_monitoring.py\model_monitoring.py]8;;\:]8;id=494087;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model_monitoring.py#1560\1560]8;;\
                             wine-monitoring-schedule                                                              

Great job! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you configured Model Monitor to watch a simple model. Next, we'll monitor the same deployment for explainability.

__REMINDER:__ Don't leave your model deployed overnight. If you aren't going to follow up with the Clarify exercise within a few hours, use the code below to remove your model:

In [ ]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()

In [ ]:
xgb_predictor.delete_endpoint()

## Clarify

For the last exercise we'll deploy an explainability monitor using Clarify. We're going to use the model that you deployed in the last exercise, but if you cleaned up your deployments from the previous exercise, that's ok! You can rerun the deployment from the previous exercise up to the point where we deployed our model. It'll look like this:

```python
xgb_predictor = model.deploy(
    initial_instance_count=1, instance_type='ml.m4.xlarge',
    data_capture_config=data_capture_config
)
```

Once your model is deployed, you can come back here. _REMINDER_: you need to clean up your deployment, don't leave it running overnight. We'll provide some code at the end to delete your deployment.

## Prep

We'll begin by reloading our data from the previous exercise.

In [20]:
data = datasets.load_wine()
df = pd.DataFrame(data['data'])
df.columns = data['feature_names']
df.rename(columns = {'od280/od315_of_diluted_wines':'od280_od315_of_diluted_wines'}, inplace=True)

We also need to put the target variable in the first column per the docs for our chosen algorithm: https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost.html

In [21]:
df["TARGET"] = data['target']
df.set_index(df.pop('TARGET'), inplace=True)
df.reset_index(inplace=True)

Now we'll upload the data to S3 as train and validation data:

In [22]:
delimiter = int(len(df)/2)
train, test = df.iloc[delimiter:], df.iloc[:delimiter]

train.to_csv("train.csv", header=False, index=False)
test.to_csv("validation.csv", header=False, index=False)

val_location = session.upload_data('./validation.csv', key_prefix="data")
train_location = session.upload_data('./train.csv', key_prefix="data")

s3_input_train = sagemaker.inputs.TrainingInput(s3_data=train_location, content_type='csv')
s3_input_validation = sagemaker.inputs.TrainingInput(s3_data=val_location, content_type='csv')

Great! Our data is staged and our model is deployed - let's monitor it for explainability. We need to define three config objects, the `SHAPConfig`, the `ModelConfig`, and the `ExplainabilityAnalysisConfig`. Below, we provide the `SHAPConfig`.

In [23]:
shap_config = sagemaker.clarify.SHAPConfig(
    baseline=[train.mean().astype(int).to_list()[1:]],
    num_samples=int(train.size),
    agg_method="mean_abs",
    save_local_shap_values=False,
)

Next up, fill in the blanks to define the `ModelConfig` and `ExplainabilityAnalysisConfig`.

In [24]:
# TODO
model_config = sagemaker.clarify.ModelConfig(
    model_name=xgb_predictor.endpoint_name,
    instance_count=1,
    instance_type='ml.m4.xlarge',
    content_type="text/csv",
    accept_type="text/csv",
)

analysis_config = sagemaker.model_monitor.ExplainabilityAnalysisConfig(
        explainability_config=shap_config,
        model_config=model_config,
        headers=train.columns.to_list()[1:],
    )

Before we apply our config, we need to create the monitor object. This is what we'll apply all our config to.

In [25]:
model_explainability_monitor = sagemaker.model_monitor.ModelExplainabilityMonitor(
    role=role,
    sagemaker_session=session,
    max_runtime_in_seconds=1800,
)

[03/29/25 06:24:10] INFO     Ignoring unnecessary instance type: None.                            ]8;id=612021;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py\image_uris.py]8;;\:]8;id=487255;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/image_uris.py#530\530]8;;\

Everything's ready! Below, create a monitoring schedule using the configs we created. Set the schedule to run _daily_.

In [26]:
# TODO 
from sagemaker.model_monitor import CronExpressionGenerator


explainability_uri = f"s3://{bucket}/model_explainability"
model_explainability_monitor.create_monitoring_schedule(
    output_s3_uri=explainability_uri,
    analysis_config=analysis_config,
    endpoint_input=xgb_predictor.endpoint_name,
    schedule_cron_expression=CronExpressionGenerator.hourly(),
)

[03/29/25 06:24:52] INFO     Uploading analysis config to {s3_uri}.                 ]8;id=559417;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/clarify_model_monitoring.py\clarify_model_monitoring.py]8;;\:]8;id=570715;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/clarify_model_monitoring.py#227\227]8;;\

[03/29/25 06:24:53] INFO     Creating Monitoring Schedule with name:                       ]8;id=272893;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model_monitoring.py\model_monitoring.py]8;;\:]8;id=194425;file:///home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/sagemaker/model_monitor/model_monitoring.py#1560\1560]8;;\
                             monitoring-schedule-2025-03-29-06-24-52-634                                           

Way to go! You can check that your schedule was created by selecting the `SageMaker components and registries` tab on the far left.

In this exercise you deployed a monitor for explainability to your SageMaker endpoint. This is the last exercise - you'll apply these learnings again in your Project at the end of the course.



__REMINDER:__ Don't leave your model deployed overnight. Use the code below to remove your model:

In [126]:
monitors = xgb_predictor.list_monitors()
for monitor in monitors:
    monitor.delete_monitoring_schedule()


Deleting Monitoring Schedule with name: monitoring-schedule-2021-09-13-17-25-08-560

Deleting Monitoring Schedule with name: wine-monitoring-schedule


In [127]:
xgb_predictor.delete_endpoint()